## Trích xuất embedding mặt

In [1]:
import insightface
import cv2
import numpy as np
import os

# Khởi tạo mô hình InsightFace với CPU
model = insightface.app.FaceAnalysis()
model.prepare(ctx_id=-1)  # Sử dụng CPU (ctx_id=0 sẽ dùng GPU nếu có)

# Hàm trích xuất embedding của khuôn mặt từ một ảnh


def extract_face_embedding(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image: {image_path}")
        return []

    # Nhận diện khuôn mặt trong ảnh
    faces = model.get(img)
    face_embeddings = []
    for face in faces:
        face_embeddings.append(face.embedding)

    return face_embeddings

d:\anaconda\envs\insightface_cnn\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Dao Thi Huyen/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Dao Thi Huyen/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Dao Thi Huyen/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Dao Thi Huyen/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Dao Thi Huyen/.insightface\models\buffalo_l\w600k_r50.onn

## Trích xuất embedding của cảnh

In [2]:
import torch
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn

# Hàm trích xuất embedding của cảnh (scene) từ một ảnh


def extract_scene_embedding(image_path):
    # Đọc ảnh
    img = Image.open(image_path).convert('RGB')

    # Cấu hình tiền xử lý ảnh cho EfficientNet-B0
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                             0.229, 0.224, 0.225]),
    ])
    img_tensor = preprocess(img).unsqueeze(0)

    # Tải mô hình EfficientNet-B0 đã được huấn luyện sẵn
    model = models.efficientnet_b0(pretrained=True)
    model.classifier = nn.Identity()  # bỏ lớp phân loại
    model.eval()

    # Trích xuất đặc trưng của cảnh
    with torch.no_grad():
        scene_embedding = model(img_tensor).numpy().squeeze()

    return scene_embedding

## Combine embedding face and scene

In [3]:
def combine_face_and_scene_embeddings(face_embeddings, scene_embedding):
    combined_embeddings = []

    for face_embedding in face_embeddings:
        # Kết hợp embedding của khuôn mặt với embedding của cảnh
        combined_embedding = np.concatenate([face_embedding, scene_embedding])
        combined_embeddings.append(combined_embedding)

    return np.array(combined_embeddings)

## Save combine_embeddings

In [24]:
%pip install faiss-cpu

  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
    --------------------------------------- 0.3/13.8 MB ? eta -:--:--
   ----- ---------------------------------- 1.8/13.8 MB 5.9 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/13.8 MB 5.9 MB/s eta 0:00:02
   ----------- ---------------------------- 3.9/13.8 MB 5.7 MB/s eta 0:00:02
   ------------ --------------------------- 4.5/13.8 MB 4.8 MB/s eta 0:00:02
   ------------- -------------------------- 4.7/13.8 MB 4.3 MB/s eta 0:00:03
   --------------- ------------------------ 5.2/13.8 MB 3.8 MB/s eta 0:00:03
   ---------------- ----------------------- 5.8/13.8 MB 3.6 MB/s eta 0:00:03
   ----------------- ---------------------- 6.0/13.8 MB 3.4 MB/s eta 0:00:03
   ------------------ --------------------- 6.6/13.8 MB 3.2 MB/s eta 0:00:03
   -------------------- ------------------- 7.1/13.8 MB 3.1 MB/s eta 0:00:03
   -------------------

In [98]:
import json


def process_images_in_folder(image_folder, scene_folder):
    all_combined_embeddings = []
    metadata = []

    for imdb_id in os.listdir(image_folder):
        print(f"đang xử lý phim {imdb_id}")
        movie_folder = os.path.join(image_folder, imdb_id)
        scene_movie_folder = os.path.join(scene_folder, imdb_id)

        if os.path.isdir(movie_folder) and os.path.isdir(scene_movie_folder):
            # Trích xuất embedding khuôn mặt

            for image_name in os.listdir(movie_folder):
                image_path = os.path.join(movie_folder, image_name)
                scene_image_path = os.path.join(scene_movie_folder, image_name)

                face_embeddings = extract_face_embedding(image_path)
                scene_embedding = extract_scene_embedding(scene_image_path)
                combine_embeddings = combine_face_and_scene_embeddings(
                    face_embeddings, scene_embedding)
                all_combined_embeddings.extend(combine_embeddings)

                for _ in combine_embeddings:
                    metadata.append({
                        'imdb_id': imdb_id,
                        'image_name': image_name,
                    })

    # Lưu metadata vào file JSON
    with open('metadata.json', 'w') as jsonfile:
        json.dump(metadata, jsonfile)
    return np.array(all_combined_embeddings)

In [99]:
all_combined_embeddings = process_images_in_folder(
    'image5_2', 'image5_scene')

đang xử lý phim 0068646


d:\anaconda\envs\InsightFace\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
d:\anaconda\envs\InsightFace\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\anaconda\envs\InsightFace\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B

đang xử lý phim 0119217
đang xử lý phim 0120082
đang xử lý phim 0120338
đang xử lý phim 0167260
đang xử lý phim 0264235
đang xử lý phim 0285331
đang xử lý phim 0285333
đang xử lý phim 0367279
đang xử lý phim 0372784
đang xử lý phim 0381061
đang xử lý phim 0407887
đang xử lý phim 0468569
đang xử lý phim 0780504
đang xử lý phim 0816692
đang xử lý phim 0848228
đang xử lý phim 0993846
đang xử lý phim 10155688
đang xử lý phim 10640346
đang xử lý phim 10648342
đang xử lý phim 1104001
đang xử lý phim 12037194
đang xử lý phim 12361974
đang xử lý phim 1345836
đang xử lý phim 1386697
đang xử lý phim 1392190
đang xử lý phim 1502407
đang xử lý phim 1517268
đang xử lý phim 1570728
đang xử lý phim 1596363
đang xử lý phim 1655389
đang xử lý phim 1706620
đang xử lý phim 1707386
đang xử lý phim 1837576
đang xử lý phim 1856101
đang xử lý phim 2015381
đang xử lý phim 2304589
đang xử lý phim 2356777
đang xử lý phim 2382320
đang xử lý phim 2488496
đang xử lý phim 2584384
đang xử lý phim 2935510
đang xử lý 

In [100]:
print(all_combined_embeddings.shape)

(19008, 1792)


In [ ]:
np.save('all_combined_embeddings.npy', all_combined_embeddings)

## Lưu combine embedding tập test

In [184]:
import json


def process_images_in_folder_test(image_folder, scene_folder):
    all_combined_embeddings = []
    metadata = []

    for imdb_id in os.listdir(image_folder):
        print(f"đang xử lý phim {imdb_id}")
        movie_folder = os.path.join(image_folder, imdb_id)
        scene_movie_folder = os.path.join(scene_folder, imdb_id)

        if os.path.isdir(movie_folder) and os.path.isdir(scene_movie_folder):
            # Trích xuất embedding khuôn mặt

            for image_name in os.listdir(movie_folder):
                image_path = os.path.join(movie_folder, image_name)
                scene_image_path = os.path.join(scene_movie_folder, image_name)

                face_embeddings = extract_face_embedding(image_path)
                scene_embedding = extract_scene_embedding(scene_image_path)
                combine_embeddings = combine_face_and_scene_embeddings(
                    face_embeddings, scene_embedding)
                all_combined_embeddings.append(
                    combine_embeddings)    # sửa extend thành append

                metadata.append({
                    'imdb_id': imdb_id,
                    'image_name': image_name,
                })

    # Lưu metadata vào file JSON
    with open('metadata_test.json', 'w') as jsonfile:
        json.dump(metadata, jsonfile)
    return np.array(all_combined_embeddings, dtype=object)

In [185]:
test_all_combined_embeddings = process_images_in_folder_test(
    'image_val', 'image_val_scene')

đang xử lý phim 0068646


d:\anaconda\envs\InsightFace\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
d:\anaconda\envs\InsightFace\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\anaconda\envs\InsightFace\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B

đang xử lý phim 0119217
đang xử lý phim 0120082
đang xử lý phim 0120338
đang xử lý phim 0167260
đang xử lý phim 0264235
đang xử lý phim 0285331
đang xử lý phim 0285333
đang xử lý phim 0367279
đang xử lý phim 0372784
đang xử lý phim 0381061
đang xử lý phim 0407887
đang xử lý phim 0468569
đang xử lý phim 0780504
đang xử lý phim 0816692
đang xử lý phim 0848228
đang xử lý phim 0993846
đang xử lý phim 10155688
đang xử lý phim 10640346
đang xử lý phim 10648342
đang xử lý phim 1104001
đang xử lý phim 12037194
đang xử lý phim 12361974
đang xử lý phim 1345836
đang xử lý phim 1386697
đang xử lý phim 1392190
đang xử lý phim 1502407
đang xử lý phim 1517268
đang xử lý phim 1570728
đang xử lý phim 1596363
đang xử lý phim 1655389
đang xử lý phim 1706620
đang xử lý phim 1707386
đang xử lý phim 1837576
đang xử lý phim 1856101
đang xử lý phim 2015381
đang xử lý phim 2304589
đang xử lý phim 2356777
đang xử lý phim 2382320
đang xử lý phim 2488496
đang xử lý phim 2584384
đang xử lý phim 2935510
đang xử lý 

In [186]:
print(test_all_combined_embeddings.shape)
print(test_all_combined_embeddings[6].shape)

(1998,)
(3, 1792)


In [187]:
print(np.linalg.norm(test_all_combined_embeddings[0][0]))

28.41461


In [188]:
np.save('test_all_combinecombine_embeddings.npy', test_all_combined_embeddings)

## truy vấn 10 ảnh giống ảnh test

In [4]:
import json

# Đọc metadata từ file JSON
with open('metadata.json', 'r') as jsonfile:
    metadata = json.load(jsonfile)


# Đọc lại embeddings từ file .npy
all_combined_embeddings = np.load('all_combined_embeddings.npy')

In [5]:
with open('metadata_test.json', 'r') as jsonfile2:
    metadata_test = json.load(jsonfile2)

test_all_combined_embeddings = np.load(
    'test_all_combinecombine_embeddings.npy', allow_pickle=True)

In [6]:
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


def find_top_similar_images(query_vectors, all_embeddings, metadata, top_n=10):
    # Lưu danh sách top kết quả từ tất cả query_vector
    all_top_results = []

    for q_vec in query_vectors:
        # Tính cosine similarity
        similarities = cosine_similarity([q_vec], all_embeddings)[0]

        # Lấy chỉ số của top N vector tốt nhất
        top_indices = np.argsort(similarities)[::-1][:top_n]

        # Lưu cả chỉ số và độ tương tự
        for idx in top_indices:
            all_top_results.append((idx, similarities[idx]))

    # Sắp xếp toàn bộ danh sách theo cosine similarity giảm dần
    all_top_results = sorted(all_top_results, key=lambda x: x[1], reverse=True)

    # Lấy top N tốt nhất từ danh sách tổng hợp
    final_top_n = all_top_results[:top_n]

    # Tách chỉ số và cosine similarity
    top_indices = [idx for idx, _ in final_top_n]

    # Lấy thông tin metadata tương ứng
    top_metadata = [metadata[i] for i in top_indices]
    imdb_ids = [meta["imdb_id"] for meta in top_metadata]
    predict_id = max(set(imdb_ids), key=imdb_ids.count)

    return predict_id

In [7]:
import os
import shutil


def save_false_image(metadata_test, idx, image_folder, false_image_folder):
    # Lấy thông tin về ảnh từ metadata_test
    imdb_id = metadata_test[idx]["imdb_id"]
    image_name = metadata_test[idx]["image_name"]
    image_path = os.path.join(image_folder, imdb_id, image_name)

    if os.path.exists(image_path):
        # Tạo thư mục cho ảnh sai (false_image) nếu chưa tồn tại
        false_image_dir = os.path.join(false_image_folder, imdb_id)
        if not os.path.exists(false_image_dir):
            os.makedirs(false_image_dir)

        # Sao chép ảnh vào thư mục false_image
        shutil.copy(image_path, os.path.join(false_image_dir, image_name))
        print(f"Image {image_name} saved to {false_image_dir}")
    else:
        print(f"Image {image_name} not found in {image_path}")

In [8]:
def calculate_accuracy(test_all_combined_embeddings, all_embeddings, metadata, metadata_test, top_n=10):
    correct_predictions = 0

    for idx, query_vector in enumerate(test_all_combined_embeddings):
        # Gọi hàm find_top_similar_images để lấy top N vecto giống nhất và dự đoán
        predict_id = find_top_similar_images(
            query_vector, all_embeddings, metadata, top_n)

        # So sánh với imdb_id thực tế từ metadata_test
        true_id = metadata_test[idx]["imdb_id"]
        if predict_id == true_id:
            correct_predictions += 1
        else:
            save_false_image(metadata_test, idx, 'image_val', 'false_image')

    # Tính độ chính xác (accuracy)
    accuracy = correct_predictions / len(test_all_combined_embeddings)
    return accuracy

In [9]:
accuracy = calculate_accuracy(test_all_combined_embeddings,
                              all_combined_embeddings, metadata, metadata_test, top_n=10)

# In kết quả
print(f"Accuracy: {accuracy * 100:.2f}%")

Image Al_Pacino_MV5BZDcwODg5ODYtZjhjOC00ZDcwLWFkN2EtYzMyZDVhM2JmZDE0XkEyXkFqcGc@._V1_QL75_UX2000_.jpg saved to false_image\0068646
Image Marlon_Brando_Al_Pacino_MV5BNWY3NTRiODMtOTdmZS00MDcyLWE5NDItOTM1YjAzNTgyYTlmXkEyXkFqcGc@._V1_QL75_UX2000_.jpg saved to false_image\0068646
Image Marlon_Brando_James_Caan_MV5BNzAxNWU2ZDMtMzZmMC00NWQ0LWEyMzItNmE2ZjEzOTc4MjZiXkEyXkFqcGc@._V1_QL75_UX2000_.jpg saved to false_image\0068646
Image Marlon_Brando_MV5BMTI3MzI3MTcwOV5BMl5BanBnXkFtZTYwNjk5MDI2._V1_QL75_UX2000_.jpg saved to false_image\0068646
Image Marlon_Brando_MV5BMTIzNDY0NTEyMF5BMl5BanBnXkFtZTYwODM0NTQ2._V1_QL75_UX2000_.jpg saved to false_image\0068646
Image Leonardo_DiCaprio_Kate_Winslet_MV5BMjA1MDQyODU4Nl5BMl5BanBnXkFtZTcwMTc2NTk1Nw@@._V1_QL75_UX2000_.jpg saved to false_image\0120338
Image Leonardo_DiCaprio_Kate_Winslet_MV5BZDRjNTI1MmQtMzIxMy00MDkyLWEzZjktNmI5MWI0N2I1ZGZmXkEyXkFqcGc@._V1_QL75_UX2000_.jpg saved to false_image\0120338
Image Cheryl_Hines_MV5BMTllZThiOWMtZjFjMC00MzA5LThmNTQtMjU0Y

## Đếm số lượng ảnh sai mỗi thư mục

In [10]:
import os


def count_images_in_folders(false_image_folder):
    # Lưu kết quả số lượng ảnh trong các thư mục con
    image_counts = {}

    # Duyệt qua tất cả các thư mục con trong thư mục false_image
    for imdb_id in os.listdir(false_image_folder):
        imdb_id_folder = os.path.join(false_image_folder, imdb_id)

        # Kiểm tra nếu là thư mục
        if os.path.isdir(imdb_id_folder):
            # Liệt kê tất cả các file ảnh trong thư mục con
            image_files = [f for f in os.listdir(imdb_id_folder) if f.lower().endswith(
                ('jpg', 'jpeg', 'png', 'gif', 'bmp'))]

            # Lưu số lượng ảnh vào dictionary
            image_counts[imdb_id] = len(image_files)

    return image_counts

In [11]:
false_image_folder = "false_image"  # Đường dẫn tới thư mục false_image

# Gọi hàm để liệt kê số lượng ảnh trong các thư mục
image_counts = count_images_in_folders(false_image_folder)

# In kết quả
for imdb_id, count in image_counts.items():
    print(f"IMDB ID: {imdb_id}, Number of images: {count}")

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
IMDB ID: 0068646, Number of images: 5
IMDB ID: 0120338, Number of images: 2
IMDB ID: 0264235, Number of images: 4
IMDB ID: 0285331, Number of images: 2
IMDB ID: 0285333, Number of images: 5
IMDB ID: 0367279, Number of images: 1
IMDB ID: 0372784, Number of images: 8
IMDB ID: 0381061, Number of images: 2
IMDB ID: 0407887, Number of images: 6
IMDB ID: 0468569, Number of images: 7
IMDB ID: 0848228, Number of images: 4
IMDB ID: 10155688, Number of images: 2
IMDB ID: 10640346, Number of images: 3
IMDB ID: 10648342, Number of images: 3
IMDB ID: 1104001, Number of images: 2
IMDB ID: 12037194, Number of images: 1
IMDB ID: 12361974, Number of images: 2
IMDB ID: 1345836, Number of images: 13
IMDB ID: 1386697, Number of images: 5
IMDB ID: 1392190, Number of images: 3
IMDB ID: 1502407, Number of images: 3
IMDB ID: 1517268, Number of images: 3
IMDB ID: 1570728